In [2]:
!ls
# Numerical Imporst 
import pandas as pd
import numpy as np
import scipy 

# Plotting 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# %matplotlib notebook

# Python 
import os

# sklearn 
from sklearn.metrics import f1_score # f1_score(y_true, y_pred)
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectFromModel

# scipy
from scipy.cluster import hierarchy as hc # for dendograms 



final_after_na.ipynb  Full-Copy4.ipynb	Full.ipynb     test_pro.tsv
final.ipynb	      Full-Copy5.ipynb	NA.ipynb       train_cli.tsv
Full-Copy10.ipynb     Full-Copy6.ipynb	README.txt     train_pro.tsv
Full-Copy1.ipynb      Full-Copy7.ipynb	start.ipynb
Full-Copy2.ipynb      Full-Copy8.ipynb	sum_tab_1.csv
Full-Copy3.ipynb      Full-Copy9.ipynb	test_cli.tsv


In [3]:
df_train_pro = pd.read_table(f'{os.getcwd()}/train_pro.tsv', 
                           delim_whitespace=True,
                           low_memory=False,).T
df_test_pro = pd.read_table(f'{os.getcwd()}/test_pro.tsv', 
                           delim_whitespace=True,
                           low_memory=False,).T
df_train_cli = pd.read_csv(f'{os.getcwd()}/train_cli.tsv', 
                           delim_whitespace=True,
                           low_memory=False,)
df_test_cli = pd.read_csv(f'{os.getcwd()}/test_cli.tsv', 
                           delim_whitespace=True,
                           low_memory=False,)
df_train_mislabel = pd.read_csv(f'{os.getcwd()}/sum_tab_1.csv', 
                           low_memory=False,)
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)
# df_train_pro
# df_test_pro
# df_train_cli
# df_test_cli
# df_train_mislabel

In [4]:
# Come back to the way you handle this NA, sophisticated way will imporve by at least 5% 
train_pro = df_train_pro.copy(deep=True)
# train_pro = train_pro.fillna(train_pro.mean())
train_pro.index.name = 'sample'


test_pro = df_test_pro.copy(deep=True)
# test_pro = test_pro.fillna(test_pro.mean())
test_pro.index.name = 'sample'

# df_train_pro
# train_pro
# df_test_pro
# test_pro

In [5]:
train_cli = df_train_cli.copy(deep=True)
train_cli = train_cli.set_index('sample')
train_cli = train_cli.replace({'gender': {'Male':0, 'Female':1},
                              'msi': {'MSI-Low/MSS':0, 'MSI-High':1}})

test_cli = df_test_cli.copy(deep=True)
test_cli = test_cli.set_index('sample')
test_cli = test_cli.replace({'gender': {'Male':0, 'Female':1},
                              'msi': {'MSI-Low/MSS':0, 'MSI-High':1}})
# df_train_cli
# train_cli
# df_test_cli
# test_cli

In [6]:
train_mislabel = df_train_mislabel.copy(deep=True)
train_mislabel = train_mislabel.set_index('sample')

# df_train_mislabel
# train_mislabel

In [7]:
train_pro.reset_index(drop=True, inplace=True)
train_cli.reset_index(drop=True, inplace=True)
train_mislabel.reset_index(drop=True, inplace=True)
# train_pro
# train_cli
# train_mislabel
train_combined = pd.concat([train_mislabel, train_cli, train_pro], axis=1)
# train_combined

### final parameters based on the importance on full dataset 

1. ANP32E                         0.202813
2. msi                            0.122349
3. AGRN                           0.104420
4. RAD50                          0.087882
5. CASK                           0.066713
6. MYO5B                          0.065782
7. XPNPEP1                        0.063634
8. FMNL1                          0.049491
9. CES2                           0.046209
10. GBP2                          0.031256
11. KLC4                          0.031149
12. ARFIP1                        0.030896
13. gender                        0.023883


In [14]:
train_final2 = train_combined [['gender',
                                'ANP32E', 
                                'AGRN', 
                                'RAD50', 
                                'CASK', 
                                'MYO5B', 
                                'XPNPEP1', 
                                'FMNL1', 
                                'CES2', 
                                'GBP2',
                                'KLC4', 
                                'ARFIP1',
                                'msi']].isnull().sum(axis = 0)


# columns = train_final2.columns

# train_final2
# train_combined

In [15]:
X_final2_train, X_final2_valid, y_final2_train, y_final2_valid = train_test_split(train_final2.values.astype(int),
                                                                                  train_combined.mismatch.values, 
                                                                                  test_size=0.3)

ValueError: Found input variables with inconsistent numbers of samples: [13, 80]

In [16]:
final2_forest = RandomForestClassifier(n_estimators=500, 
                                      min_samples_leaf=5,
                                      max_features=None,
                                      n_jobs=-1,
                                      oob_score=True)

final2_forest.fit(X_final2_train, y_final2_train)

final2_importances = final2_forest.feature_importances_
final2_indices = np.argsort(final2_importances)[::-1]

# for f in range(X_train.shape[1]): # to get the full results
for f in range(len(columns)):
    print("%2d) %-*s %f" % (f + 1, 30, columns[final2_indices[f]], final2_importances[final2_indices[f]]))
    
plt.title('Feature Importance')
plt.bar(range(X_final2_train.shape[1]), final2_importances[final2_indices], align='center')

NameError: name 'X_final2_train' is not defined

In [17]:
y_final2_pred = final2_forest.predict(X_final2_valid)
print('Random Forest: \n', classification_report(y_true=y_final2_valid, y_pred=y_final2_pred))
print('OOB score: ', final2_forest.oob_score_)

NameError: name 'X_final2_valid' is not defined

In [18]:
final2_select = SelectFromModel(final2_forest, threshold=0.05)

final2_select.fit(X_final2_train, y_final2_train)

X_final2_important_train = final2_select.transform(X_final2_train)
X_final2_important_train_df = pd.DataFrame(X_final2_important_train)
X_final2_important_valid = final2_select.transform(X_final2_valid)
final2_forest.fit(X_final2_important_train, y_final2_train)
y_final2_pred = final2_forest.predict(X_final2_important_valid)
print('Random Forest: \n', classification_report(y_true=y_final2_valid, y_pred=y_final2_pred))
print('OOB score: ', final2_forest.oob_score_)

NameError: name 'X_final2_train' is not defined

In [19]:
final2_corr = np.round(scipy.stats.spearmanr(X_final2_important_train).correlation, 4)
final2_corr_condensed = hc.distance.squareform(1-final2_corr)
final2_z = hc.linkage(final2_corr_condensed, method='average')
fig = plt.figure(figsize=(16,10))
final2_dendrogram = hc.dendrogram(final2_z, labels=X_final2_important_train_df.columns, orientation='left', leaf_font_size=16)
plt.show()

NameError: name 'X_final2_important_train' is not defined

In [8]:
train_combined[['gender','ANP32E', 'PPP1CC', 'AGRN', 'RAD50',  'CASK', 'MYO5B', 'XPNPEP1', 'FMNL1', 'CES2', 'GBP2', 'KLC4', 'ARFIP1', 'msi']].isnull().sum(axis = 0)

gender      0
ANP32E      0
PPP1CC      7
AGRN        2
RAD50       2
CASK        2
MYO5B       7
XPNPEP1     0
FMNL1      14
CES2       13
GBP2       14
KLC4        2
ARFIP1      2
msi         0
dtype: int64

### msi
1. CASK                           0.064384
2. ADPGK                          0.061092
3. ANP32E                         0.048545
4. KLC4                           0.040509
5. TAP1                           0.038269
6. NCAPG                          0.037576
7. RAD50                          0.035045
8. FMNL1                          0.033152
9. GBP2                           0.030121
10. XPNPEP1                        0.029699
11. PTPRC                          0.025719

### gender
1. PPP1CC                         0.050713
2. AGRN                           0.037314
4. ARFIP1                         0.029830
5. CES2                           0.023406
6. MYO5B                          0.021966
